In [1]:
import os

# Define the paths
project_path = r'C:\PROFESSIONAL\RIST\7th Sem\Minor_Project_Details\Test_1'
train_path = os.path.join(project_path, 'train')
valid_path = os.path.join(project_path, 'valid')
test_path = os.path.join(project_path, 'test')
data_yaml_path = os.path.join(project_path, 'data.yaml')

Training Model 

In [ ]:
from ultralytics import YOLO

# Load YOLOv8 model (for example, yolov8n.pt which is the nano version)
model = YOLO('yolov8n.pt')  # or 'yolov8s.pt' for a slightly larger version

# Train the model using the data.yaml file
results = model.train(
    data=r'C:\PROFESSIONAL\RIST\7th Sem\Minor_Project_Details\METHOD\1_YOLO8\Test_1\data.yaml',  # Path to your data.yaml
    epochs=50,  # Set the number of training epochs
    imgsz=224,  # Image size (default is 640x640)
    batch=16,   # Batch size
    name='leaf_detection'  # Folder name for saving the model
)


Check Output predicted

In [ ]:
import cv2
from ultralytics import YOLO
import torch
import os
# Specify the input and output folders
input_folder = '"C:/PROFESSIONAL/RIST/7th Sem/Minor_Project_Details/METHOD/1_YOLO8/Test_1/inputTest"'  # Update with your folder containing images
output_folder = '"C:/PROFESSIONAL/RIST/7th Sem/Minor_Project_Details/METHOD/1_YOLO8/Test_1/outputTest"'  # Update with your desired output folder


# Check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

# Load the custom-trained YOLO model for leaf detection
model = YOLO('runs/detect/leaf_detection/weights/best.pt')  # Update with your model path
model.to(device)

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process each image in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):  # Adjust extensions if needed
        # Load the image
        img_path = os.path.join(input_folder, filename)
        image = cv2.imread(img_path)
        
        if image is None:
            print(f"Error loading image {filename}")
            continue
        
        # Get image dimensions
        img_height, img_width = image.shape[:2]
        
        # Run inference on the image
        results = model(image, device=device)

        # Extract and annotate detections
        for result in results:
            boxes = result.boxes.xyxy.cpu().numpy()  # Bounding box coordinates
            class_ids = result.boxes.cls.cpu().numpy()  # Class IDs
            confidences = result.boxes.conf.cpu().numpy()  # Confidence scores

            for box, class_id, confidence in zip(boxes, class_ids, confidences):
                x1, y1, x2, y2 = map(int, box)
                label = model.names[int(class_id)]  # Get the class label
                label_text = f'{label} {confidence:.2f}'

                # Set color for bounding box
                color = (0, 255, 0)  # Green color for bounding boxes
                cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)

                # Determine text size and position it inside the bounding box
                (text_width, text_height), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
                text_x = x1
                text_y = y1 - 5 if y1 - 5 > text_height else y1 + text_height + 5  # Adjust to fit within image
                
                # Ensure text doesn't go outside the image
                if text_x + text_width > img_width:
                    text_x = img_width - text_width - 10

                # Draw the label text
                cv2.rectangle(image, (text_x, text_y - text_height - 2), (text_x + text_width, text_y + 2), color, -1)
                cv2.putText(image, label_text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        # Save the annotated image to the output folder
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, image)
        print(f"Processed and saved: {output_path}")

print("Leaf detection completed for all images.")


ROC graph

In [ ]:
import numpy as np
from sklearn.metrics import roc_curve, auc, accuracy_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize

# Sample data for 5 classes
true_labels = [0, 1, 2, 1, 0, 2, 3, 4]  # Replace with actual true class labels
predicted_labels = [0, 1, 1, 1, 0, 2, 3, 4]  # Replace with actual predicted class labels
class_names = ['Bohera', 'Devilbackbone', 'Nayontara', 'Neem', 'Pathorkuchi']

# Calculate accuracy and F1 score
accuracy = accuracy_score(true_labels, predicted_labels)
print(f'Accuracy: {accuracy:.2f}')

f1 = f1_score(true_labels, predicted_labels, average='weighted')
print(f'F1 Score: {f1:.2f}')

efficiency = accuracy  # In classification, efficiency is often taken as accuracy
print(f'Efficiency: {efficiency:.2f}')

# Classification Report
print("\nClassification Report:")
print(classification_report(true_labels, predicted_labels, target_names=class_names))

# ROC Curve for Multi-class
true_labels_binarized = label_binarize(true_labels, classes=range(len(class_names)))
predicted_labels_binarized = label_binarize(predicted_labels, classes=range(len(class_names)))
n_classes = len(class_names)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(true_labels_binarized[:, i], predicted_labels_binarized[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot the ROC curve for each class
plt.figure()
for i, class_name in enumerate(class_names):
    plt.plot(fpr[i], tpr[i], label=f'ROC curve of {class_name} (area = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--')  # Dashed diagonal
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Leaf Detection Classes')
plt.legend(loc='lower right')
plt.show()


In [ ]:
import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize

# Sample data
true_labels = [0, 1, 2, 1, 0, 2, 3, 4, 5, 6, 7]  # Replace with actual true class labels
predicted_labels = [0, 1, 1, 1, 0, 2, 3, 4, 5, 6, 7]  # Replace with actual predicted class labels
class_names = ['Bohera', 'Devilbackbone', 'Nayontara', 'Neem', 'Pathorkuchi',]

# Binarize the output
true_labels_binarized = label_binarize(true_labels, classes=range(len(class_names)))
predicted_labels_binarized = label_binarize(predicted_labels, classes=range(len(class_names)))
n_classes = len(class_names)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(true_labels_binarized[:, i], predicted_labels_binarized[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot the ROC curve for each class
plt.figure()
for i, class_name in enumerate(class_names):
    plt.plot(fpr[i], tpr[i], label=f'ROC curve of {class_name} (area = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--')  # Dashed diagonal
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Leaf Detection Classes')
plt.legend(loc='lower right')

# Save the ROC curve as an image
plt.savefig('roc_curve_leaf_detection.png', format='png', dpi=300)  # Save with high resolution
plt.show()


<Figure size 640x480 with 1 Axes>